# Output Parsers
- LLMの出力をプログラム的に扱う補助をしてくれる
- 出力形式を指定したLLMの応答から該当箇所を抽出してPythonオブジェクトに変換してくれる

## PydanticOutputParser

---
#### pydantic
- データクラス(データの入れ物)を定義できるパッケージ
- データ定義、バリデーション、シリアライズ・デシリアライズ（JSONとの相互変換等）が簡単にできる

In [1]:
from pydantic import BaseModel, Field

# ★BaseModelを継承したクラスを定義
class Recipe(BaseModel):
    ingredients: list[str] = Field(description='料理の材料')  # ★クラス変数: 型指定 でメンバ変数を定義
    steps: list[str] = Field(description='手順')

In [2]:
# こんな感じでオブジェクトを作れる
recipe = Recipe(
    ingredients=['いちご', 'みかん', 'りんご'],
    steps=['材料を切る', '材料を混ぜる', '材料を盛る'],
)

recipe

Recipe(ingredients=['いちご', 'みかん', 'りんご'], steps=['材料を切る', '材料を混ぜる', '材料を盛る'])

In [3]:
# JSON化
json_str = recipe.json()
print(json_str)

# インデントや改行を入れるには少し工夫が必要
import json
json_str = json.dumps(recipe.model_dump(), indent=2, ensure_ascii=False)
print('\n\n' + json_str)

{"ingredients":["いちご","みかん","りんご"],"steps":["材料を切る","材料を混ぜる","材料を盛る"]}


{
  "ingredients": [
    "いちご",
    "みかん",
    "りんご"
  ],
  "steps": [
    "材料を切る",
    "材料を混ぜる",
    "材料を盛る"
  ]
}


In [4]:
# JSONからの作成も簡単
json_data = """
{
  "ingredients": [
    "いちご",
    "みかん",
    "りんご"
  ],
  "steps": [
    "材料を切る",
    "材料を混ぜる",
    "材料を盛る"
  ]
}
"""
Recipe.parse_raw(json_data)

Recipe(ingredients=['いちご', 'みかん', 'りんご'], steps=['材料を切る', '材料を混ぜる', '材料を盛る'])

---
#### LangChainでの活用
- PydanticのBaseModelを継承したデータクラスを作る
- データクラスをもとに、プロンプトで出力形式を指定する文字列を生成
- LLMの出力からデータクラスのオブジェクトに変換

In [5]:
# ★現状 pydanticの最新バージョンは2だが、PydanticOutputParserはバージョン1にしか対応していない
from pydantic.v1.main import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

# ★Pydanticバージョン1 の BaseModelを継承したクラスを定義
class Recipe(BaseModel):
    ingredients: list[str] = Field(description='料理の材料')  # ★クラス変数: 型指定 でメンバ変数を定義
    steps: list[str] = Field(description='手順')

# パーサーを作成
parser = PydanticOutputParser(pydantic_object=Recipe)  # ★定義したクラスを指定

# プロンプトに埋め込む出力形式指定の文字列を取得
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "\u6599\u7406\u306e\u6750\u6599", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "\u624b\u9806", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```


In [6]:
from langchain.prompts import PromptTemplate

# プロンプト作成
template = """
料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['dish'],
    partial_variables={'format_instructions': format_instructions},
)

formatted_prompt = prompt.format(dish='カレー')
print(formatted_prompt)


料理のレシピを教えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "\u6599\u7406\u306e\u6750\u6599", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "\u624b\u9806", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```

料理名: カレー



In [7]:
# LLMに投げる
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

messages = [
    HumanMessage(content=formatted_prompt),
]

result = chat(messages)
print(result.content)

/usr/local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{
  "ingredients": [
    "玉ねぎ",
    "人参",
    "じゃがいも",
    "牛肉",
    "カレールー",
    "水"
  ],
  "steps": [
    "1. 材料を準備する。",
    "2. 玉ねぎ、人参、じゃがいもを切る。",
    "3. 牛肉を炒める。",
    "4. 野菜を加えて炒める。",
    "5. 水を加えて煮込む。",
    "6. カレールーを溶かして加える。",
    "7. 煮込んで完成。"
  ]
}


In [8]:
# Pydanticのモデルインスタンスに変換
recipe = parser.parse(result.content)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['玉ねぎ', '人参', 'じゃがいも', '牛肉', 'カレールー', '水'] steps=['1. 材料を準備する。', '2. 玉ねぎ、人参、じゃがいもを切る。', '3. 牛肉を炒める。', '4. 野菜を加えて炒める。', '5. 水を加えて煮込む。', '6. カレールーを溶かして加える。', '7. 煮込んで完成。']


#### 注意点
- 必ず変換可能な形式で返ってくるわけではない
- ではどうするか？
    - エラーハンドリングする
    - LLMにテキスト修正してもらう OutputFixingParser, PetryWithErrorOutputParser 等を使う
    - Function Callingを使って指定形式での出力確率を高める